In [1]:
data_var = ''

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
5214,2024-02-19,Eua Ncaa,22:30,Texas Southern,Southern,1.88,1.88,138.5,1.88,1.81,-1.5,1.92,1.69,G4Q76zmO,0.531915,0.531915,0.531915,0.552486,0.063830,122.722,32.427032,0.264232,1.8,1.643168,0.912871,99.54,1.708,0.251933,0.147501,8.0,124.226,27.936087,0.224881,3.0,0.000000,0.000000,136.29,1.712,0.370095,0.216177,34.0,63,77,1.58,1.77,95.030,132.200,0.000000,0.026828,0.090102,-1.14,-0.228,-3.859649,0.000000,0.0,0.000000,0.82,0.164,5.365854,0.451094,0.4,-0.051094
5215,2024-02-19,Israel Wbl Feminina,14:00,Hapoel Rishon LeZion F,Elitzur Ramla F,11.50,1.01,152.5,1.87,1.74,17.5,2.07,1.63,rqMCsqZI,0.086957,0.990099,0.534759,0.574713,0.077056,176.054,149.897573,0.851430,1.2,1.643168,1.369306,85.88,2.758,2.435091,0.882919,37.0,98.802,6.073934,0.061476,3.0,0.000000,0.000000,99.51,1.166,0.095289,0.081723,102.0,76,93,1.13,1.07,0.000,881.974,1.185859,0.050927,0.168177,0.00,0.000,inf,0.000000,0.0,0.000000,39.71,7.942,0.001259,0.000000,0.0,0.000000
5216,2024-02-19,Eua Ncaa,23:00,Houston,Iowa State,1.18,5.05,129.5,1.92,1.75,-10.5,1.97,1.69,pnTO1PHK,0.847458,0.198020,0.520833,0.571429,0.045477,101.428,14.456558,0.142530,2.4,1.341641,0.559017,97.58,1.396,0.309806,0.221924,33.0,131.220,29.890045,0.227786,2.4,1.341641,0.559017,106.60,1.852,0.512221,0.276577,32.0,82,82,1.19,1.30,80.732,160.750,0.878492,0.065509,0.108191,0.45,0.090,2.000000,0.000000,0.0,0.000000,1.50,0.300,13.500000,0.000000,0.0,0.000000
5217,2024-02-19,Eua Ncaa,23:00,Norfolk State,N.Carolina Central,1.50,2.59,135.5,1.88,1.81,-5.5,1.97,1.69,02ogbflO,0.666667,0.386100,0.531915,0.552486,0.052767,82.186,8.815749,0.107266,2.4,1.341641,0.559017,88.75,1.188,0.078230,0.065851,16.0,117.716,47.975461,0.407553,1.8,1.643168,0.912871,198.44,1.596,0.537801,0.336968,27.0,71,82,1.25,2.42,108.682,126.972,0.376893,0.026828,0.108191,2.16,0.432,1.157407,0.000000,0.0,0.000000,-0.16,-0.032,-49.687500,0.442391,0.5,0.057609
5218,2024-02-19,Eua Ncaa,23:00,Texas,Kansas State,1.19,4.69,139.5,1.88,1.88,-10.5,2.07,1.65,OxSS0qXQ,0.840336,0.213220,0.531915,0.531915,0.053556,183.912,79.498957,0.432266,1.2,1.643168,1.369306,299.51,2.634,1.444102,0.548254,17.0,186.098,105.368775,0.566200,0.6,1.341641,2.236068,131.76,2.742,1.557264,0.567930,-27.0,61,72,4.91,1.83,129.502,288.666,0.841794,0.000000,0.159669,-2.05,-0.410,-0.463415,0.744917,0.6,-0.144917,-5.00,-1.000,-3.690000,0.000000,0.0,0.000000
5219,2024-02-19,Israel Wbl Feminina,14:00,Asa Jerusalem F,Holon F,2.55,1.45,140.5,1.82,1.82,3.5,1.99,1.68,fNkVaQlm,0.392157,0.689655,0.549451,0.549451,0.081812,502.504,193.081702,0.384239,0.0,0.000000,NaN,437.90,8.686,3.072488,0.353729,-135.0,510.230,728.944833,1.428659,0.6,1.341641,2.236068,1794.00,7.760,10.439016,1.345234,-69.0,58,69,7.55,26.00,0.000,0.000,0.388909,0.000000,0.119457,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
5220,2024-02-19,Israel Wbl Feminina,14:00,Hapoel Jerusalem F,Hapoel Petah Tikva F,1.07,7.33,148.5,1.88,1.74,-16.5,2.03,1.66,GWjR06Zt,0.934579,0.136426,0.531915,0.574713,0.071005,211.746,115.295231,0.544498,2.4,1.341641,0.559017,171.77,2.378,1.097301,0.461439,78.0,286.514,233.331698,0.814381,1.2,1.643168,1.369306,83.46,4.456,3.408787,0.764988,-41.0,89,78,1.93,1.07,208.208,0.000,1.053926,0.054693,0.141805,0.69,0.138,0.507246,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
5228,22:30,Eua Ncaa,Prairie View A&M,Grambling St.,1.84,Back Home
